In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append("../")
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.leaves.parametric.Parametric import Bernoulli
try:
    from time import perf_counter
except:
    from time import time as perf_counter
from spn.io.Graphics import *
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from spn.algorithms.MPE import mpe
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.Sampling import sample_instances
from numpy.random.mtrand import RandomState
from scipy.stats import pearsonr
from copy import deepcopy





print("Reading dataframes \n")
df_train = pd.read_csv('train.csv')
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = pd.read_csv('test.csv')
df_test = df_test.sample(frac=1).reset_index(drop=True)

data_train = df_train.values
data_test = df_test.values



def binning(data_array):
    
    instances, features = data_array.shape
    dark = np.zeros([instances, features])
    mean_array = np.mean(data_array, axis = 0, dtype = np.float64)
    
    for row in range(instances):
        for feature in range(features):
            
            if data_array[row][feature] >= mean_array[feature]:
                dark[row][feature]  = 1
            else:
                dark[row][feature] = 0
                
    return dark


print("Binning data \n")
train_data = binning(data_train)
test_data = binning(data_test)
assert train_data.shape[0]
eagle_train = train_data.astype(int)
eagle_test = test_data.astype(int)
#eagle_valid = eaglez10[test_size:,:].astype(np.int)
assert eagle_test.shape[1] == eagle_train.shape[1] 
assert eagle_test.shape[0] != eagle_train.shape[0]
np.savetxt('light.ts.data', eagle_train, fmt = '%1d', delimiter = ",")
np.savetxt('light.test.data', eagle_test, fmt = '%1d', delimiter = ",")
del df_train
del df_test    


print("Learning SPN \n")
name = 'light'
train = np.loadtxt( name + ".ts.data", dtype=float, delimiter=",", skiprows=0)
test = np.loadtxt( name + ".test.data", dtype=float, delimiter=",", skiprows=0)
D = np.vstack((train, test))
F = D.shape[1]
features = ["V" + str(i) for i in range(F)]
name.upper()
ds_context = Context(parametric_types= np.asarray([Bernoulli] * F)).add_domains(train)
eval_start_t = perf_counter()
spn = learn_parametric(train, ds_context, min_instances_slice=20)
eval_end_t = perf_counter()
print('Structure of SPN learnt in %f secs' % (eval_end_t - eval_start_t))  
print("............................................\n\n")




print("Plotting SPN \n")
def plot_network(spn, fname="spn.png"):

    import networkx as nx
    from networkx.drawing.nx_pydot import graphviz_layout
    import matplotlib.pyplot as plt
    plt.clf()
    g, labels = get_networkx_obj(spn)
    pos = graphviz_layout(g, prog='dot')
    #plt.figure(figsize=(200, 100))
    ax = plt.gca()
    ax.invert_yaxis()
    nx.draw(g, pos, with_labels=True, arrows=False, node_color='lightgrey', edge_color='grey', width=1, node_size=1,
            labels=labels, font_size=1)
    ax.collections[0].set_edgecolor("#333333")
    #edge_labels = nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=nx.get_edge_attributes(g, 'weight'), font_size=1,
                                              # clip_on=False, alpha=0.6)
    xpos = list(map(lambda p: p[0], pos.values()))
    ypos = list(map(lambda p: p[1], pos.values()))
    ax.set_xlim(min(xpos)-20,max(xpos)+20)
    ax.set_ylim(min(ypos)-20,max(ypos)+20)
    plt.tight_layout()
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig(fname,bbox_inches='tight', pad_inches = 0)
    
def plot_network_with_weights(spn, fname="spnWeights.png"):
    import networkx as nx
    from networkx.drawing.nx_pydot import graphviz_layout
    import matplotlib.pyplot as plt
    plt.clf()
    g, labels = get_networkx_obj(spn)
    pos = graphviz_layout(g, prog='dot')
    plt.figure(figsize=(200, 100))
    ax = plt.gca()
    ax.invert_yaxis()
    nx.draw(g, pos, with_labels=True, arrows=False, node_color='lightgrey', edge_color='grey', width=2, node_size=1,
            labels=labels, font_size=4)
    ax.collections[0].set_edgecolor("#333333")
    edge_labels = nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=nx.get_edge_attributes(g, 'weight'), font_size=1,
                                               clip_on=False, alpha=0.6)
    xpos = list(map(lambda p: p[0], pos.values()))
    ypos = list(map(lambda p: p[1], pos.values()))
    ax.set_xlim(min(xpos)-20,max(xpos)+20)
    ax.set_ylim(min(ypos)-20,max(ypos)+20)
    plt.tight_layout()
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig(fname,bbox_inches='tight', pad_inches = 0)
plot_network(spn)
plot_network_with_weights(spn)






print('Running Inference \n')
test_copy = deepcopy(test)
test_copy[:,0:5] = np.nan
sampled_data = sample_instances(spn, test_copy, RandomState(123))
mpe_data = mpe(spn, test_copy)


print("............................................\n")
print('\n Sampled Data ------->')
print("NumofSubhalos : MSE %f,  R_2 %f ,  Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 0], sampled_data[:, 0]), \
 r2_score(test[:, 0], sampled_data[:, 0]), \
 accuracy_score(test[:, 0], sampled_data[:, 0]), \
 pearsonr(test[:, 0], sampled_data[:, 0])[0]))
print("Central Black Hole Mass : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 1], sampled_data[:, 1]), \
 r2_score(test[:, 1], sampled_data[:, 1]), \
 accuracy_score(test[:, 1], sampled_data[:, 1]), \
 pearsonr(test[:, 1], sampled_data[:, 1])[0]))
print("Stellar Mass  : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 2], sampled_data[:, 2]), \
 r2_score(test[:, 2], sampled_data[:, 2]), \
 accuracy_score(test[:, 2], sampled_data[:, 2]), \
 pearsonr(test[:, 2], sampled_data[:, 2])[0]))
print("Velocity Dispersion : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 3], sampled_data[:, 3]), \
 r2_score(test[:, 3], sampled_data[:, 3]), \
 accuracy_score(test[:, 3], sampled_data[:, 3]), \
 pearsonr(test[:, 3], sampled_data[:, 3])[0]))
print("Cg Vmax : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 4], sampled_data[:, 4]), \
 r2_score(test[:, 4], sampled_data[:, 4]), \
 accuracy_score(test[:, 4], sampled_data[:, 4]), \
 pearsonr(test[:, 4], sampled_data[:, 4])[0]))




print("..............")
print('\n MPE Data ------>')
print("NumofSubhalos : MSE %f,  R_2 %f ,  Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 0], mpe_data[:, 0]), \
 r2_score(test[:, 0], mpe_data[:, 0]), \
 accuracy_score(test[:, 0], mpe_data[:, 0]), \
 pearsonr(test[:, 0], mpe_data[:, 0])[0]))
print("Central Black Hole Mass : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 1], mpe_data[:, 1]), \
 r2_score(test[:, 1], mpe_data[:, 1]), \
 accuracy_score(test[:, 1], mpe_data[:, 1]), \
 pearsonr(test[:, 1], mpe_data[:, 1])[0]))
print("Stellar Mass  : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 2], mpe_data[:, 2]), \
 r2_score(test[:, 2], mpe_data[:, 2]), \
 accuracy_score(test[:, 2], mpe_data[:, 2]), \
 pearsonr(test[:, 2], mpe_data[:, 2])[0]))
print("Velocity Dispersion : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 3], mpe_data[:, 3]), \
 r2_score(test[:, 3], mpe_data[:, 3]), \
 accuracy_score(test[:, 3], mpe_data[:, 3]), \
 pearsonr(test[:, 3], mpe_data[:, 3])[0]))
print("Cg Vmax : MSE %f, R_2 %f , Accuray_Score %f , pearsonr %f \n"% ( \
 mean_squared_error(test[:, 4], mpe_data[:, 4]), \
 r2_score(test[:, 4], mpe_data[:, 4]), \
 accuracy_score(test[:, 4], mpe_data[:, 4]), \
 pearsonr(test[:, 4], mpe_data[:, 4])[0]))


Reading dataframes 

Binning data 

Learning SPN 

Structure of SPN learnt in 144.725214 secs
............................................


Plotting SPN 

Running Inference 

............................................


 Sampled Data ------->
NumofSubhalos : MSE 0.125649,  R_2 -0.418729 ,  Accuray_Score 0.874351 , pearsonr 0.489249 

Central Black Hole Mass : MSE 0.093294, R_2 -0.245904 , Accuray_Score 0.906706 , pearsonr 0.426679 

Stellar Mass  : MSE 0.043757, R_2 0.390498 , Accuray_Score 0.956243 , pearsonr 0.652172 

Velocity Dispersion : MSE 0.240421, R_2 -0.092744 , Accuray_Score 0.759579 , pearsonr 0.437230 

Cg Vmax : MSE 0.166279, R_2 0.182816 , Accuray_Score 0.833721 , pearsonr 0.582622 

..............

 MPE Data ------>
NumofSubhalos : MSE 0.038050,  R_2 0.570369 ,  Accuray_Score 0.961950 , pearsonr 0.771995 

Central Black Hole Mass : MSE 0.039717, R_2 0.469593 , Accuray_Score 0.960283 , pearsonr 0.735701 

Stellar Mass  : MSE 0.019542, R_2 0.727792 , Accuray_Score 0.98

<Figure size 432x288 with 0 Axes>